In [0]:
!pip install pandas
!pip install tweepy
!pip install vaderSentiment

In [0]:
import tweepy
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [0]:
#My Twitter API Authentication Variables
consumer_key = 'QKwJwo8eh5xId499X3Tp5dpjn'
consumer_secret = 'xvtwLpJkSprYfvHzstVhjAsPIxUOSVD7aG0MLTvNpBfBDCi70G'
access_token = '279879713-2XjuGwajgWcraqYM70J40dekg1p9mQQDtNA4PJGN'
access_token_secret = 'tbkplFclgGR4b8TsitNNK2l7e11DYkx3MX2HHCOeBLWDN'

In [51]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

tweets = api.search('India', count=200)


data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

display(data.head(10))




,Tweets
0,RT @INCIndia: We are deeply relieved to hear t...
1,RT @ChJiyaa: India me kn c sim pr acha interne...
2,RT @ArvindChirania: #FightAgainstTerrorism Whi...
3,RT @Naby_SZN: I don't have any affiliation wit...
4,RT @aajtak: पाकिस्तान ने भारतीय विमानों को मार...
5,RT @CBhattacharji: @VishnuNDTV @sunetrac @soni...
6,RT @chathootti: Lost decade. Only development ...
7,RT @theworldindex: Share of worldwide plastic ...
8,RT @GeorgeFulton1: This pisses me off. After t...
9,RT @iihtishamm: Imran Khan should take his dec...


In [23]:
print(tweets[0].lang)

en


In [26]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [0]:
sid = SentimentIntensityAnalyzer()


listy = []
sentiment = []

for index, row in data.iterrows():
  ss = sid.polarity_scores(row["Tweets"])
  #print(ss)
  
  for i in ss:
    if i=='neg':
      if ss[i] > 0.5:
        polarity = 'negative'
    elif i=='neu':
      if ss[i] > 0.5:
        polarity = 'neutral'
    elif i=='pos':
      if ss[i] > 0.5:
        polarity = 'positive'
    else:
      if ss[i] > 100:
        polarity = 'Compound'
    
  
  listy.append(ss)
  sentiment.append(polarity)
  


In [35]:
#polarity = null
for i in ss:
  if i=='neg':
    if ss[i] > 0.5:
      polarity = 'negative'
  elif i=='neu':
    if ss[i] > 0.5:
      polarity = 'neutral'
  elif i=='pos':
    if ss[i] > 0.5:
      polarity = 'positive'
  else:
    if ss[i] > 0.5:
      polarity = 'Compound'
    
  print(i, ss[i])
  
polarity
  

neg 0.118
neu 0.642
pos 0.241
compound 0.3182


'neutral'

In [53]:
se = pd.Series(listy)
se1 = pd.Series(sentiment)
data['polarity'] = se.values
data['Sentiment'] = se1.values

display(data.head(100))

,Tweets,polarity,Sentiment
0,RT @INCIndia: We are deeply relieved to hear t...,"{'neg': 0.0, 'neu': 0.888, 'pos': 0.112, 'comp...",neutral
1,RT @ChJiyaa: India me kn c sim pr acha interne...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
2,RT @ArvindChirania: #FightAgainstTerrorism Whi...,"{'neg': 0.0, 'neu': 0.802, 'pos': 0.198, 'comp...",neutral
3,RT @Naby_SZN: I don't have any affiliation wit...,"{'neg': 0.169, 'neu': 0.769, 'pos': 0.062, 'co...",neutral
4,RT @aajtak: पाकिस्तान ने भारतीय विमानों को मार...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
5,RT @CBhattacharji: @VishnuNDTV @sunetrac @soni...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
6,RT @chathootti: Lost decade. Only development ...,"{'neg': 0.172, 'neu': 0.674, 'pos': 0.154, 'co...",neutral
7,RT @theworldindex: Share of worldwide plastic ...,"{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp...",neutral
8,RT @GeorgeFulton1: This pisses me off. After t...,"{'neg': 0.119, 'neu': 0.881, 'pos': 0.0, 'comp...",neutral
9,RT @iihtishamm: Imran Khan should take his dec...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral


In [54]:
data['Sentiment'].describe()

count         100
unique          1
top       neutral
freq          100
Name: Sentiment, dtype: object